<a href="https://colab.research.google.com/github/SaqlainHussainShah/Cross-site-scripting-attacks/blob/master/XSS_V2_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
# from xml.dom import minidom

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 

import tensorflow as tf
from tensorflow import keras
import os
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
df = pd.read_csv('./XSS.csv', encoding='utf-8-sig')

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16317 entries, 0 to 16316
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  16317 non-null  int64 
 1   Sentence    16317 non-null  object
 2   Label       16317 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 382.6+ KB


In [0]:
df=df[df.columns[-2:]]  # Only get sentence and labels

In [64]:


sentences=df['Sentence'].values
sentences[1]



'<abbr oncut="alert(1)" contenteditable>test</abbr>'

In [0]:
labels = df['Label'].values

In [66]:
sentences[:30]

array(["['canfranc' 8 'norm' 'norm']",
       '<abbr oncut="alert(1)" contenteditable>test</abbr>',
       '<picture onmouseover="alert(1)">test</picture>',
       '<div role="navigation" class="navbox" aria-labelledby="Emerging_technologies_in_information_and_communications" style="padding:3px"><table class="nowraplinks hlist mw-collapsible autocollapse navbox-inner" style="border-spacing:0;background:transparent;color:inherit"><tbody><tr><th scope="col" class="navbox-title" colspan="2" style="text-align: center;">',
       '<li id="cite_note-FOOTNOTEDomingos2015210-207"><span class="mw-cite-backlink"><b><a href="#cite_ref-FOOTNOTEDomingos2015210_207-0">^ </a> </b> ',
       '<li><cite id="CITEREFMoravec1988" class="citation book"><a href="/wiki/Hans_Moravec" title="Hans Moravec">Moravec, Hans </a> (1988). <span class="cs1-lock-registration" title="Free registration required"><a rel="nofollow" class="external text" href="https://archive.org/details/mindchildrenfutu00mora"><i>Mind Chil

In [0]:


def preprocess_xss_data(data):
    
    for i in range(len(data)):
        
        data[i]=data[i].replace('<', ' lessthan ')
        data[i]=data[i].replace('>', ' greaterthan ')
        data[i]=data[i].replace('/', ' forwardslash ')
        data[i]=data[i].replace('(', ' roundbracketstart ')
        data[i]=data[i].replace(')', ' roundbracketend ')
        data[i]=data[i].replace('=', ' = ')
    
    return data



In [0]:
sentences = preprocess_xss_data(sentences)

In [69]:
len(sentences)

16317

In [70]:
sentences[:30]

array(["['canfranc' 8 'norm' 'norm']",
       ' lessthan abbr oncut = "alert roundbracketstart 1 roundbracketend " contenteditable greaterthan test lessthan  forwardslash abbr greaterthan ',
       ' lessthan picture onmouseover = "alert roundbracketstart 1 roundbracketend " greaterthan test lessthan  forwardslash picture greaterthan ',
       ' lessthan div role = "navigation" class = "navbox" aria-labelledby = "Emerging_technologies_in_information_and_communications" style = "padding:3px" greaterthan  lessthan table class = "nowraplinks hlist mw-collapsible autocollapse navbox-inner" style = "border-spacing:0;background:transparent;color:inherit" greaterthan  lessthan tbody greaterthan  lessthan tr greaterthan  lessthan th scope = "col" class = "navbox-title" colspan = "2" style = "text-align: center;" greaterthan ',
       ' lessthan li id = "cite_note-FOOTNOTEDomingos2015210-207" greaterthan  lessthan span class = "mw-cite-backlink" greaterthan  lessthan b greaterthan  lessthan a h

In [71]:
preprocessed_df.head()

,Sentences,Label
0,['canfranc' 8 'norm' 'norm'],0
1,"lessthan abbr oncut = ""alert roundbracketstar...",1
2,"lessthan picture onmouseover = ""alert roundbr...",1
3,"lessthan div role = ""navigation"" class = ""nav...",0
4,"lessthan li id = ""cite_note-FOOTNOTEDomingos2...",0


In [72]:
df.head()

,Sentence,Label
0,['canfranc' 8 'norm' 'norm'],0
1,"lessthan abbr oncut = ""alert roundbracketstar...",1
2,"lessthan picture onmouseover = ""alert roundbr...",1
3,"lessthan div role = ""navigation"" class = ""nav...",0
4,"lessthan li id = ""cite_note-FOOTNOTEDomingos2...",0


In [0]:
df.dropna(inplace=True)

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16317 entries, 0 to 16316
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  16317 non-null  object
 1   Label     16317 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 382.4+ KB


In [0]:


oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token=oov_tok, lower=False,split=' ')

# tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)



In [0]:
total_words = len(tokenizer.word_index) + 1

In [77]:
total_words


13773

In [0]:
import numpy as np

In [0]:
input_sequences = []
for line in sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [80]:
len(sentences)

16317

In [81]:
len(input_sequences)

16317

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [0]:


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers



In [89]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len)) # Your Embedding Layer)
model.add(Bidirectional(LSTM(150, return_sequences=True)))# An LSTM Layer)
# model.add(Dropout(0.2)) # A dropout layer)
model.add(LSTM(100))# Another LSTM Layer)
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))# A Dense Layer including regularizers)
model.add(Dense(1,activation='sigmoid'))# A Dense Layer)
# Pick an optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])# Pick a loss function and an optimizer)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1229, 100)         1377300   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1229, 300)         301200    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               51712     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 513       
Total params: 1,891,125
Trainable params: 1,891,125
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
input_sequences = np.array(input_sequences,dtype='float64')
labels = np.array(labels, dtype ='float64')

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(input_sequences, labels, test_size=0.2, random_state=42)

In [93]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=64)

Epoch 1/10
204/204 [==============================] - 65s 318ms/step - loss: 0.3902 - accuracy: 0.9689 - val_loss: 0.0558 - val_accuracy: 0.9917
Epoch 2/10
204/204 [==============================] - 64s 313ms/step - loss: 0.0403 - accuracy: 0.9935 - val_loss: 0.0421 - val_accuracy: 0.9936
Epoch 3/10
204/204 [==============================] - 64s 313ms/step - loss: 0.0362 - accuracy: 0.9947 - val_loss: 0.0487 - val_accuracy: 0.9914
Epoch 4/10
204/204 [==============================] - 64s 313ms/step - loss: 0.0335 - accuracy: 0.9948 - val_loss: 0.0462 - val_accuracy: 0.9933
Epoch 5/10
204/204 [==============================] - 63s 308ms/step - loss: 0.0315 - accuracy: 0.9953 - val_loss: 0.0377 - val_accuracy: 0.9939
Epoch 6/10
204/204 [==============================] - 63s 310ms/step - loss: 0.0339 - accuracy: 0.9943 - val_loss: 0.0397 - val_accuracy: 0.9923
Epoch 7/10
204/204 [==============================] - 63s 310ms/step - loss: 0.0292 - accuracy: 0.9950 - val_loss: 0.0447 - val_ac

In [0]:
pred = model.predict(X_test)

In [0]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [0]:
from sklearn.metrics import accuracy_score

In [97]:
accuracy_score(y_test,pred)

0.9917279411764706

In [0]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [0]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [0]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [0]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1.0:
            if pred == 1.0:
                true_positive += 1
            elif pred != 1.0:
                false_negative += 1

        elif true == 0.0:
            if pred == 0.0:
                true_negative += 1
            elif pred != 0.0:
                false_positive += 1
       
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [0]:
accuracy,precision,recall=confusion_matrix(y_test,pred)

In [103]:
print(" For Embedding \n Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For Embedding 
 Accuracy : 0.9917279411764706 
 Precision : 0.9859241126070991 
 Recall : 0.9975232198142415


In [104]:

model.save('model_embedding.h5')

Using TensorFlow backend.


In [0]:
from keras.models import load_model

mymodel = tf.keras.models.load_model('model_embedding.h5')

In [0]:
import numpy as np
import cv2
def predict_cross_site_script():
    
    repeat=True
    
    
    
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)
    
    if input_val== '0':
        repeat=False
        
        
    
    if repeat == True:

        # // Preprocess
        data = input_val
        
        data=data.replace('<', ' lessthan ')
        data=data.replace('>', ' greaterthan ')
        data=data.replace('/', ' forwardslash ')
        data=data.replace('(', ' roundbracketstart ')
        data=data.replace(')', ' roundbracketend ')
        data=data.replace('=', ' = ')

        token_list = tokenizer.texts_to_sequences([data])[0]

        input_sequences = []
        
        for i in range(1, len(token_list)):
          n_gram_sequence = token_list[:i+1]

        input_sequences.append(n_gram_sequence)

        input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

        prediction=mymodel.predict(input_sequences)

        if prediction>0.5:

    #         print(f"Chances of attack :  {prediction[0]*100} ")
            print(" It can be Cross site scripting attack")

        else:

    #         print(f" Chances of being safe {100 - (prediction*100) }")
            print("seems safe")
        
        
        predict_cross_site_script()
        
        
    elif repeat == False:
        print( " Good Bye ")

In [114]:
predict_cross_site_script()

Give me some data to work on : As salam o Alaikum
seems safe
Give me some data to work on : <table oncopy="alert(1)" contenteditable>test</table>
 It can be Cross site scripting attack
Give me some data to work on : ['semicopado' 10 'norm' 'norm']
seems safe
Give me some data to work on : ['<SCRIPT FOR=document EVENT=onreadystatechange>javascript:alert(1)</SCRIPT>']
 It can be Cross site scripting attack
Give me some data to work on : <iframe oncopy="alert(1)" contenteditable>test</iframe>
 It can be Cross site scripting attack
Give me some data to work on : <li class="toclevel-2 tocsection-28"><a href="#Logic"><span class="tocnumber">6.2 
seems safe
Give me some data to work on : 0
 Good Bye 
